# project

## project 的三个需求

1. 表示数字艺术品
2. 存储数字艺术品
3. 交换数字艺术品 transaction

## 安全需求分析

1. 需要记录用户和用户拥有的艺术品
    - 需要将用户和用户的独特签名对应起来
    - 需要在艺术品的存储中添加用户的独特签名
    - 将签名添加到艺术品的算法需要加密，防止攻击者修改艺术品数据内的独特签名
2. 数据交换

    - 交换艺术品时，需要将交换的请求加密，防止被中途修改
    - ACID 原则
        - 原子性（A）：一个事务的所有系列操作步骤被看成一个动作，所有的步骤要么全部完成，要么一个也不会完成。如果在事务过程中发生错误，则会回滚到事务开始前的状态，将要被改变的数据库记录不会被改变。
        - 一致性（C）：一致性是指在事务开始之前和事务结束以后，数据库的完整性约束没有被破坏，即数据库事务不能破坏关系数据的完整性及业务逻辑上的一致性。
        - 隔离性（I）：主要用于实现并发控制，隔离能够确保并发执行的事务按顺序一个接一个地执行。通过隔离，一个未完成事务不会影响另外一个未完成事务。
        - 持久性（D）：一旦一个事务被提交，它应该持久保存，不会因为与其他操作冲突而取消这个事务。

3. 数据存储的安全性

    - NFT 的不可篡改性依靠区块链的安全特性来实现。这里，我们将区块链替换为了一般的数据库，这将使平台失去去中心化的优势。为了仍然保障安全性，需要做出以下假设：
        - 部署数据库的设备不会被攻破
        - 数据库管理员的密码不会被泄露
        - 数据库管理员不会恶意修改数据库

4. 需要保护所有用户的独特签名

## 重要 reference

-   [中泰证券 NFT 技术分析](https://dfscdn.dfcfw.com/download/A2_cms_f_20220216123508144922&direct=1&abc3847.pdf)
-   [我的总结（祥见参考列表） - csdn](https://blog.csdn.net/weixin_39591031/article/details/124138855)


In [ ]:
import sqlite3
import json
import base64
import io
import time
import typing

from Crypto import Random
from Crypto.PublicKey import RSA
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.Random import get_random_bytes

from PIL import Image


### 数据库

处理一切和数据库的交互

reference:

-   [https://developer.51cto.com/article/624601.html](https://developer.51cto.com/article/624601.html)
-   [https://www.runoob.com/sqlite/sqlite-data-types.html](https://www.runoob.com/sqlite/sqlite-data-types.html)


In [ ]:
class DBmanager:
    DATABASE_PATH = "./demo.db"
    COLLECTIONS_TABLE_NAME = "collections"
    USER_TABLE_NAME = "users"
    transactionS_TABLE_NAME = "transactions"

    def __init__(self):
        # init connection, db will be created if doesnt exist
        self.conn = sqlite3.connect(self.DATABASE_PATH)
        self.cur = self.conn.cursor()
        # init tables
        self.init_collections_table()
        self.init_user_table()
        self.init_transactions_table()

    # init tables

    def init_collections_table(self):
        if (
            len(
                self.cur.execute(
                    "SELECT name FROM sqlite_master WHERE type='table' AND name='{}';".format(
                        self.COLLECTIONS_TABLE_NAME
                    )
                ).fetchall()
            )
            > 0
        ):
            print("Find '{}' table in db.".format(self.COLLECTIONS_TABLE_NAME))
            return
        # id | owner_id | price | encrypted_content | preview | statue
        self.cur.execute(
            "CREATE TABLE '{}' (ID TEXT, OWNER_ID TEXT, PRICE REAL, ENCRYPTED_CONTENT BOLB, PREVIEW BOLB, STATUS TEXT);".format(
                self.COLLECTIONS_TABLE_NAME
            )
        )
        self.conn.commit()
        print("Images table initialized.")

    def init_user_table(self):
        if (
            len(
                self.cur.execute(
                    "SELECT name FROM sqlite_master WHERE type='table' AND name='{}';".format(
                        self.USER_TABLE_NAME
                    )
                ).fetchall()
            )
            > 0
        ):
            print("Find '{}' table in db.".format(self.USER_TABLE_NAME))
            return
        # id | validation_file | pub_key | balance
        self.cur.execute(
            "CREATE TABLE '{}' (ID TEXT, VALIDATION_FILE TEXT, PUB_KEY TEXT, BALANCE REAL);".format(
                self.USER_TABLE_NAME
            )
        )
        self.conn.commit()
        print("Users table initialized.")

    def init_transactions_table(self):
        if (
            len(
                self.cur.execute(
                    "SELECT name FROM sqlite_master WHERE type='table' AND name='{}';".format(
                        self.transactionS_TABLE_NAME
                    )
                ).fetchall()
            )
            > 0
        ):
            print("Find '{}' table in db.".format(self.transactionS_TABLE_NAME))
            return
        # id | timestamp | type | content | collection_id | src_user_id | dest_user_id | status
        self.cur.execute(
            "CREATE TABLE '{}' (\
                ID INTEGER PRIMARY KEY, \
                TIMESTAMP REAL, \
                TYPE TEXT, \
                CONTENT TEXT, \
                COLLECTION_ID TEXT, \
                SRC_USER_ID TEXT, \
                DEST_USER_ID TEXT, \
                STATUS TEXT);".format(
                self.transactionS_TABLE_NAME
            )
        )
        self.conn.commit()
        print("Users table initialized.")

    # manage collections TABLE

    def add_collection(
        self,
        collection_id: str,
        price: float = None,
        owner_id: str = None,
        encrypted_content: str = None,
        preview: bytes = None,
        status: str = None,
    ):
        self.cur.execute(
            "INSERT INTO '{}' VALUES('{}', '{}', '{}', '{}', '{}', '{}')".format(
                self.COLLECTIONS_TABLE_NAME,
                collection_id,
                price,
                owner_id,
                encrypted_content,
                preview,
                status,
            )
        )
        self.conn.commit()

    def remove_collection(self, collection_id):
        self.cur.execute(
            "DELETE FROM '{}' WHERE id = '{}'".format(
                self.COLLECTIONS_TABLE_NAME, collection_id
            )
        )
        self.conn.commit()

    def update_collection(
        self,
        collection_id: str,
        owner_id: str = None,
        price: float = None,
        encrypted_content: bytes = None,
        preview: bytes = None,
        status: str = None,
    ):
        """Update any field of the collection table in database."""
        for field_name, field_value in zip(
            [
                f"{owner_id=}".split("=")[0],
                f"{price=}".split("=")[0],
                f"{encrypted_content=}".split("=")[0],
                f"{preview=}".split("=")[0],
                f"{status=}".split("=")[0],
            ],
            [owner_id, price, encrypted_content, preview, status],
        ): # field_name is the name of the variable
            if field_value:
                self.cur.execute(
                    "UPDATE '{}' SET '{}' = '{}' WHERE id = '{}';".format(
                        self.COLLECTIONS_TABLE_NAME,
                        field_name,
                        field_value,
                        collection_id,
                    )
                )
                self.conn.commit()

    def get_all_collections(self):
        self.cur.execute("SELECT * FROM '{}'".format(self.COLLECTIONS_TABLE_NAME))
        return self.cur.fetchall()

    def get_collection_by_id(self, collection_id):
        """
        Find collection from database. Return the collection info if exist, otherwise None.
        @return All data item of the colelction: (id, owner_id, price, encrypted_content, preview, statue)
        """
        self.cur.execute(
            "SELECT * FROM '{}' WHERE id = '{}'".format(
                self.COLLECTIONS_TABLE_NAME, collection_id
            )
        )
        res = self.cur.fetchall()
        if len(res) > 1:
            raise AssertionError("Fatel error, more than one collecion have same id.")
        return res[0]

    def get_collections_by_user_id(self, user_id):
        """
        Find all collections belongs to user. Return the collection info list.
        @return [(id, owner_id, price, encrypted_content, preview, statue), ...]
        """
        self.cur.execute(
            "SELECT * FROM '{}' WHERE owner_id = '{}'".format(
                self.COLLECTIONS_TABLE_NAME, user_id
            )
        )
        res = self.cur.fetchall()
        return res

    # manage users TABLE

    def add_user(
        self,
        user_id: str,
        validation_file: str = None,
        pub_key: str = None,
        balance: float = None,
    ):
        self.cur.execute(
            "INSERT INTO '{}' VALUES('{}', '{}', '{}', '{}')".format(
                self.USER_TABLE_NAME, user_id, validation_file, pub_key, balance,
            )
        )
        self.conn.commit()

    def remove_user(self, user_id):
        self.cur.execute(
            "DELETE FROM '{}' WHERE id = '{}'".format(self.USER_TABLE_NAME, user_id)
        )
        self.conn.commit()

    def update_user(
        self,
        user_id: str,
        validation_file: bytes = None,
        pub_key: str = None,
        balance: float = None,
    ):
        """Update any field of the collection table in database."""
        for field_name, field_value in zip(
            [
                f"{validation_file=}".split("=")[0],
                f"{pub_key=}".split("=")[0],
                f"{balance=}".split("=")[0],
            ],
            [validation_file, pub_key, balance],
        ):
            if field_value:
                self.cur.execute(
                    "UPDATE '{}' SET '{}' = '{}' WHERE id = '{}';".format(
                        self.USER_TABLE_NAME, field_name, field_value, user_id,
                    )
                )
                self.conn.commit()

    def get_all_user(self):
        self.cur.execute("SELECT * FROM '{}'".format(self.USER_TABLE_NAME))
        return self.cur.fetchall()

    def get_user_by_id(self, user_id):
        """
        Find user from database. Return the user info if exist, otherwise None.
        @return All data item of the user: (id, validation_file, pub_key, balance)
        """
        self.cur.execute(
            "SELECT * FROM '{}' WHERE id = '{}'".format(self.USER_TABLE_NAME, user_id)
        )
        res = self.cur.fetchall()
        if len(res) > 1:
            raise AssertionError("Fatel error, more than one collecion have same id.")
        return res[0]

    # manage transactions TABLE

    def add_transaction(
        self,
        timestamp: float = None,
        type: str = None,
        content: str = None,
        collection_id: str = None,
        src_user_id: str = None,
        dest_user_id: str = None,
        status: str = None,
    ):
        self.cur.execute(
            "INSERT INTO '{}' VALUES('NULL', '{}', '{}', '{}', '{}', '{}', '{}', '{}')".format(
                self.transactionS_TABLE_NAME,
                timestamp,
                type,
                content,
                collection_id,
                src_user_id,
                dest_user_id,
                status,
            )
        )
        self.conn.commit()

    def get_all_transaction(self):
        self.cur.execute("SELECT * FROM '{}'".format(self.transactionS_TABLE_NAME))
        return self.cur.fetchall()

    def get_transactions_by_user_id(self, user_id):
        """
        Find all transecitons related to user (either src_user_id or dest_user_id). Return the transeciton info list.
        @return [(id, owner_id, price, encrypted_content, preview, statue), ...]
        """
        self.cur.execute(
            "SELECT * FROM '{}' WHERE src_user_id = '{}' OR dest_user_id = '{}'".format(
                self.transactionS_TABLE_NAME, user_id, user_id
            )
        )
        res = self.cur.fetchall()
        return res

    # general function

    def destroy(self):
        self.cur.close()
        self.conn.close()
        print("Db connection closed.")


In [ ]:
# init database
db = DBmanager()


In [ ]:
%%script false
db.destroy()


### 用户

属性

-   `ID`: user name, must be unique, thus can be view as ID
-   `validation_file`: json serilized file (2 fields: user_id & AES key) being encrypted using user's RSA private key
-   `pub_key`: user's RSA public key
-   `balance`: user's balance of XAV coin
-   `transactions`: user's all transactions

TODO: 数据库条件查询语句可以在DBmanager里实现。只需要附带条件字符串参数即可。

In [ ]:
class User:
    DEFAULT_BALANCE = 3  # user default balance
    db = None

    def __init__(
        self,
        id: str,
        pub_key: str = None,
        validation_file: bytes = None,
        balance: float = None,
        collections: list = None,
        transactions: list = None,
        # must be provided in instantiation:
        db: DBmanager = None,
    ):
        """
        id: user name, must be unique, thus can be view as ID
        pub_key: user's RSA public key
        validation_file: json serilized file (2 fields: user_id & AES key) being encrypted using user's RSA private key
        balance: user's balance of XAV coin
        collections: user's all collections
        transactions: user's all transactions
        db: DBmanager
        """

        # db must be provided in instantiation
        if not (db or User.db):  # if both are None
            raise AttributeError("Haven't connect to database, please connect first.")
        elif User.db == None and db != None:  # User.db == None and db != None
            User.db = db  # assign db to User class-variable, to be used by any instance

        self.id = id
        self.pub_key = pub_key
        self.validation_file = validation_file
        self.balance = balance
        self.collections = collections or self._get_collections()
        self.transactions = transactions or self._get_transactions()

    @classmethod
    def fromID(cls, id, db: DBmanager = None):
        """Load a user by fetching the info from database using user id."""
        if not cls.if_id_exist(id, cls.db or db):
            raise AttributeError("Collection doesn't exist with id={}.".format(id))
        _, validation_file, pub_key, balance = cls.db.get_user_by_id(id)
        collections = cls.db.get_collections_by_user_id(id)
        transactions = cls.db.get_transactions_by_user_id(id)
        return cls(id, validation_file, pub_key, balance, collections, transactions, db)

    @classmethod
    def new(cls, id, db: DBmanager = None):
        """
        Create a new user with a id.
        @AttributeError raise exception if id already exist.
        @return user instance and the RSA private key of user.
        """
        if cls.if_id_exist(id, cls.db or db):
            raise AttributeError("User id already exists with id={}.".format(id))
        priv_key, pub_key, aes_key = cls._gen_keys()
        validation_file = cls.gen_validation_file(id, aes_key)
        balance = cls.DEFAULT_BALANCE
        user = cls(id, pub_key, validation_file, balance, [], [], db)
        user._add_to_db()
        return user, priv_key

    def if_id_exist(id, db: DBmanager = None):
        """Return whether or not the user's id already exists in database."""
        # db.cur.execute(
        #     "SELECT * FROM '{}' WHERE id = '{}'".format(db.USER_TABLE_NAME, id)
        # )
        if db == None and User.db == None:
            raise AttributeError("Need to provide a DBmanager")
        if db != None and User.db == None:
            User.db = db
        db.cur.execute("SELECT * FROM users WHERE id = '{}'".format(id))
        return len(db.cur.fetchall()) > 0

    def _gen_keys() -> typing.Tuple[str, str, bytes]:
        """
        Generate:
            1. a pair of keys using EEC algorithm (P-256 curve)
            2. a key using AES algorithm (CTR mode, 32-bytes random VI)
        """
        random_generator = Random.new().read
        rsa = RSA.generate(2048, random_generator)

        # generate priv key and pub key, and bytes -> str format
        priv_key = rsa.exportKey().decode('utf-8')
        pub_key = rsa.publickey().exportKey().decode('utf-8')

        # 32-bytes (256-bits) is the safest
        aes_key_bytes = get_random_bytes(32)
        # bytes are hard to directly transform to string
        aes_key = base64.b64encode(aes_key_bytes).decode("utf-8")

        return priv_key, pub_key, aes_key

    def gen_validation_file(id, pub_key, aes_key):
        data = json.dumps({"user_id": id, "aes_key": aes_key}) # str
 
        # encrypt with RSA public key
        pub_key = RSA.import_key(pub_key.encode('utf-8')) # str -> bytes
        cipher_rsa = PKCS1_OAEP.new(pub_key)
        encrypted_data_bytes = cipher_rsa.encrypt(data) # bytes
        encrypted_data = base64.b64encode(encrypted_data_bytes).decode('utf-8') # bytes -> str

        return encrypted_data

    def decrypt_validation_file(self, priv_key:str)->typing.Tuple[str, str]:
        '''
        Decrypt the validation file and return the file content.
        @return user_id contained in the validation file and user's AES key (base64 encoded string)
        '''
        encrypted_data_bytes = base64.b64decode(self.validation_file.encode('utf-8')) # str -> bytes
        priv_key = RSA.import_key(priv_key)
        cipher_rsa = PKCS1_OAEP.new(priv_key)
        decrypted_data_bytes = cipher_rsa.decrypt(encrypted_data_bytes) # bytes
        decrypted_data = decrypted_data_bytes.decode('utf-8') # bytes -> str
        data = json.loads(decrypted_data)

        user_id, aes_key = data['user_id'], data['aes_key']
        return user_id, aes_key

    def _add_to_db(self):
        """
        - id: user name, must be unique, thus can be view as ID
        - pub_key: user's RSA public key
        - validation_file: json serilized file (2 fields: user_id & AES key) being encrypted using user's RSA private key
        - balance: user's balance of XAV coin
        - transactions: user's all transactions
        """
        self.db.add_user(
            self.id, self.validation_file, self.pub_key, self.balance,
        )

    def _get_collections(self):
        # retrieve user's collections from database
        return self.db.get_collections_by_user_id(self.id)

    def _get_transactions(self):
        # retrieve user's transactions from database
        return self.db.get_transactions_by_user_id(self.id)

    def __repr__(self):
        return """User:\n\tid={}\n\tpub_key={}\n\tvalidation_file={}\n\tbalance={}\n\tcollections={}\n\ttransactions={}""".format(
            self.id,
            self.pub_key,
            self.validation_file,
            self.balance,
            self.collections,
            self.transactions,
        )


#### 测试

In [ ]:
# create first user of User class
xav, pwd = User.new(id="xav", db=db)
xav

In [ ]:
# create user without passing DBmanager
bill, bill_pwd = User.new(id='bill')
bill

In [ ]:
# test User.fromID()
xav_copy = User.fromID(xav.id)
xav_copy

### Collection

属性：
- id
- owner_id
- price: 初始价格 0.1 XAV，每次交易升值 1 XAV
- encrpted_content
- preview: 低分辨率预览图
- status

数据库格式：
`(id, owner_id, price, encrypted_content, preview, status)`

注：想要解决盗版图片重新上传的问题
1. 算法检查图片重复度
2. 审核人员人工检查


In [ ]:
class Collection:
    # Format in database: (id, owner_id, price, encrypted_content, preview, status)

    _DEFAULT_PRICE = 0.1  # default price of a collection
    _STATUS_CONFIRMED = "confirmed"  # default status
    _STATUS_PENDING = "pending"  # collection on processing
    db = None  # database

    def __init__(
        self,
        id: str,
        owner_id: str,
        price: float,
        encrypted_content: str,
        preview: str,
        status: str,
        # must be provided in instantiation:
        db: DBmanager = None,
    ):
        """
        @params
        - id: collection unique name
        - owner_id: id of collection's owner
        - price: price of the collection, auto increase by 1 after each transaction
        - encrypted_content: raw data of the collection after encrypted with owner's AES key
        - preview: low resolution version of the image
        - status: pending if in the middle of a transaction, otherwise confirmed
        - raw_data: raw data of the collection in bytes
        - ase_key: a ase key used to decrypt `encrypted_content`
        - db: DBmanager instance.
        """

        # db must be provided in instantiation
        if not (db or Collection.db):  # if both are None
            raise AttributeError("Haven't connect to database, please connect first.")
        elif Collection.db == None and db != None:
            Collection.db = db

        self.id = id
        self.owner_id = owner_id
        self.price = price
        self.encrypted_content = encrypted_content
        self.preview = preview
        self.status = status

    @classmethod
    def fromID(cls, id, db:DBmanager=None):
        '''Load a collection by fetching data from database using colleciton id.'''
        if not cls.if_id_exist(id):
            raise AttributeError("Collection doesn't exist with id={}.".format(id))
        _, owner_id, price, encrypted_content, preview, statue = cls.db.get_collection_by_id(id)
        return cls(id, owner_id, price, encrypted_content, preview, statue, db)

    @classmethod
    def new(cls, id, owner_id, raw_data, aes_key, db:DBmanager=None):
        '''Create a new collection and add to database.'''
        if cls.if_id_exist(id):
            raise AttributeError("Collection id already exists, please use another id.")
        price = cls._DEFAULT_PRICE
        encrypted_content = cls._encrypte_content(raw_data, aes_key)
        preview = cls._gen_preview(raw_data)
        status = cls._STATUS_CONFIRMED
        collection = cls(id, owner_id, price, encrypted_content, preview, status, db)
        collection._add_to_db()
        return collection

    def if_id_exist(self, id):
        """Return whether or not the collection's id already exists in database."""
        self.db.cur.execute(
            "SELECT * FROM '{}' WHERE id = '{}'".format(self.db.COLLECTIONS_TABLE_NAME, id)
        )
        return len(self.db.cur.fetchall()) > 0

    def _add_to_db(self):
        """
        - id: collection unique name
        - price: price of the collection, auto increase by 1 after each transaction
        - owner_id: id of collection's owner
        - encrypted_content: raw data of the collection after encrypted with owner's AES key
        - preview: low resolution version of the image
        - status: pending if in the middle of a transaction, otherwise confirmed
        """
        self.db.add_collection(
            self.id,
            self.price,
            self.owner_id,
            self.encrypted_content,
            self.preview,
            self.status,
        )

    def _encrypte_content(data, aes_key) -> str:
        """
        Encrypt content using AES (CTR mode, allow arbitrary length of data).
        @param data: raw data of image in bytes
        @return serialized json string (e.g., {"nonce": '4Sa\we', "ciphertext": 'wgS2F=D3'})
        """
        cipher = AES.new(aes_key, AES.MODE_CTR)
        ct_bytes = cipher.encrypt(data)
        nonce = base64.b64encode(cipher.nonce).decode("utf-8")
        ct = base64.b64encode(ct_bytes).decode("utf-8")
        result = json.dumps({"nonce": nonce, "ciphertext": ct})
        print("Encrypt result:", result)
        return result

    def _decrypte_content(data, aes_key) -> bytes:
        """
        Encrypt content using AES (CTR mode, allow arbitrary length of data).
        @param data: json serialized string (e.g., {"nonce": '4Sa\we', "ciphertext": 'wgS2F=D3'})
        @return decrypted bytes data
        """
        b64 = json.loads(data)
        nonce = base64.b64decode(b64["nonce"])
        ct = base64.b64decode(b64["ciphertext"])
        cipher = AES.new(aes_key, AES.MODE_CTR, nonce=nonce)
        pt = cipher.decrypt(ct)
        print("Decrypt result:", pt)
        return pt

    def _gen_preview(raw_data):
        '''Generate low resolution thumbnail and return its bytes data.'''
        PREVIEW_SIZE = (210, 294)  # default collection thubnail size (width, height)
        img = Image.frombytes(raw_data).thumbnail(PREVIEW_SIZE)
        img_byte_stream = io.BytesIO()
        img.save(img_byte_stream, format=img.format)
        return img_byte_stream.getvalue()

    def update_owner(self, new_owner_id: str):
        """Change owner of this artwork."""
        # update owner of this collection in database
        print("Image owner updated: '{}' ---> '{}'".format(self.owner_id, new_owner_id))
        self.db.update_collection(self.id, owner_id=new_owner_id)


### transactions

属性
- id: auto-increment id
- timestamp: the timestamp when the transaction is created.
- type
- content
- collection_id
- src_user_id
- dest_user_id
- status

In [ ]:
class transaction:
    db = None
    def __init__(self, timestamp, type, content, collection_id, src_user_id, dest_user_id, status, db:DBmanager=None):
        # db must be provided in instantiation
        if not (db or self.db):  # if both are None
            raise AttributeError("Haven't connect to database, please connect first.")
        else:
            self.db = db
            
        self.timestamp = timestamp
        self.type = type
        self.content = content
        self.collection_id = collection_id
        self.src_user_id = src_user_id
        self.dest_user_id = dest_user_id
        self.status = status
    
    @classmethod
    def new(cls, type, content, collection_id, src_user_id, dest_user_id, status):
        return cls(time.time(), type, content, collection_id, src_user_id, dest_user_id, status)

### Controller
为 UI 提供所有功能所需的API

In [ ]:
class Controller:
    def __init__(self, user_list:list, colelction_list:list, transaction_list:list):
        self._user_list = user_list
        self._collection_list = colelction_list
        self._transaction_list = transaction_list

        # do rest initialization

    def sign_up(self, user_id:str)->str:
        '''
        Sign up a user, user only need to provide a unique username as id. 
        Return user's private key if success, otherwise None.
        (Note: user_id cannot include whitespace.)
        '''
        # check whether user_id include whitespace & whether user_id is unique
        if user_id.find(' ') != -1 or User.if_id_exist(user_id, User.db):
            return None

        # create a new user, update in database and update Controller's list
        user, priv_key = User.new(user_id) # database updated here
        self._user_list.append(user)

        return priv_key

    def sign_in(self, user_id:str, priv_key:str):
        '''
        Sign in using user_id and user's private key. Return True if success.
        (Note: private key will only be stored in local variable to prevent safety issue.)
        Pre-request: User class has already been instanciated at least once.
        '''
        # check whether user_id exist in database
        if not User.if_id_exist(user_id): # ignore the db if pre-request if fulfilled
            return False

        # using priv_key decrypt user's validation file, then json.loads the string
        # content to get a json structure, if the user_id file in json structure can
        # match the user_id provided by user, then success, and return True. otherwise,
        # return false (TBD: along with failure reason)
        user = self.find_user(user_id)
        decrypted_id, _ = user.decrypt_validation_file(priv_key)
        
        if user_id != decrypted_id:
            return False
        
        return True

    def upload(self, collection_id:str, creator_id:str, raw_data:bytes, priv_key:str):
        '''
        Upload a collection to database. Return True if success.
        (Note: private key and collection's raw data will only be stored in local variable to prevent safety issue.)
        (Note: need to update display since database is updated.)
        '''
        # check whether collection id (which is also the user-chose title for the collection) is unique in database
        if Collection.if_id_exist(collection_id):
            return False
        
        # create a new collection and add to collection_list
        creator = self.find_user(creator_id)
        _, aes_key = creator.decrypt_validation_file(priv_key)
        new_collection = Collection.new(collection_id, creator_id, raw_data, aes_key)
        self._collection_list.append(new_collection)
        return True
        
    def buy(self, buyer_id:str, collection_id:str):
        '''
        User (id=buyer_id) send a buy request to the collection's owner.
        (Note: buyer's money will be reduced only when owner accept the buying request)
        '''
        # buyer send a request (transaction) to collection owner to buy the collection
        pass

    def response(self, user_id:str, status:str, priv_key:str):
        '''
        Collection owner response to a buying request. Return True if owner accept the request.
        '''
        # if owner refuse the request
        pass

        #   send a notify (transaction) to buyer and return False
        pass


        # if owner accept the request
        pass

        #   check whether buyer has enough, return False if doesn't
        pass

        #   set the collection's status to pending until buyer sign in to the platform
        pass

        #   update transaction status (e.g., clear the rest buying requests of this collection)
        pass

        #   update both user's collection_list (database & User instances)
        pass

    def recharge(self, user_id:str, amount:float):
        '''Recharge XAV coins to user's account. Return True if success.'''
        # add amount of XAV to user's account (User instance)
        pass
    
    def download(self, collection_id:str, priv_key:str):
        '''
        User downlaod the original data of one of user's collection. Return
        the collection's raw data if successful, otherwise None.
        '''
        # find the collection's validation_file and encrypted_content
        pass
        # get key from decrypting validation_file and decrypt encrypted_content to get raw_data
        pass
        # return the raw_data if success, otherwise None
        pass

    def find_user(self, user_id:str)->User:
        '''
        Find user's instance from _user_list. Return the user's instance
        if successful, otherwise None.
        '''
        # search for the user with user_id within _user_list and return it
        pass

        # return None if cannot find the user.
        pass

    def find_collection(self, collection_id:str)->Collection:
        '''
        Find user's instance from _collection_list. Return the collection's
        instance if successful, otherwise None.
        '''
        # search for the collection with collection_id within _collection_list and return it
        pass

        # return None if cannot find the collection
        pass

    def find_transaction(self, type:str=None, colelction_id:str=None, src_user_id:str=None, dest_user_id:str=None, status:str=None)->typing.List[transaction]:
        '''
        Find all the transactions that fulfill the searching requirement. Return
        all qualified transecitons in a list. List is empty if no transaction is found.
        If no requirement is given, return all transactions.
        '''
        # if all parameters are None, return all transactions
        pass

        # search for the transactions with the right field value
        pass

        # return None if cannot find qualified transaction
        pass

